In [ ]:
!pip install --upgrade langchain 

In [1]:
import os
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama

prompt = ChatPromptTemplate.from_template("{topic}에 대한 간단한 농담을 말해줘.")
model = ChatOllama(model=os.getenv("OPENAI_DEFAULT_MODEL"))
output_parser = StrOutputParser()

chain = prompt | model | output_parser # LCEL의 기본 파이프라인

chain.invoke({"topic": "아이스크림"})

'물론이죠! 여기 아이스크림에 대한 간단한 농담이 있습니다:\n\n"왜 아이스크림 콘이 학교에서 좋은 학생으로 알려져 있을까요? 왜냐고요? 그는 항상 자신의 한계를 \'설\'정하기 때문이죠!"\n\n(설정이라는 단어는 영어로 \'limit\'이나 \'boundary\'를 의미하며, 이 농담에서는 아이스크림의 양을 스스로 조절한다는 이중적인 의미를 담고 있습니다.)\n\n즐겁게 웃으셨기를 바랍니다!'

In [2]:
!pip install langchain docarray tiktoken

Defaulting to user installation because normal site-packages is not writeable


In [3]:
# pip install langchain docarray tiktoken

from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_community.chat_models import ChatOpenAI
from langchain_community.embeddings import HuggingFaceEmbeddings

vectorstore = DocArrayInMemorySearch.from_texts(
    ["해리슨은 학교에서 일합니다.", "곰은 꿀을 좋아해"],
    embedding=HuggingFaceEmbeddings(),
)
retriever = vectorstore.as_retriever()

template = """다음 지문에만 근거해서 질문에 답하세요:
{context}

질문: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOllama(model=os.getenv("OPENAI_DEFAULT_MODEL"))
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel( # 여러개의 Runnable을 병렬적으로 실행
    {"context": retriever, "question": RunnablePassthrough()} # RunnablePassthrough는 값을 입력받아 그대로 전달하는 객체
)
chain = setup_and_retrieval | prompt | model | output_parser

# retrieval | prompt | model | output. RAG의 기본 파이프라인

/tmp/ipykernel_44378/392265637.py:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding=HuggingFaceEmbeddings(),
/tmp/ipykernel_44378/392265637.py:12: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding=HuggingFaceEmbeddings(),
/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as n

In [4]:
chain.invoke("해리슨은 어디에서 일하나요?")

'제공된 문서에 따르면, 해리슨은 **학교**에서 일합니다.'

In [5]:

chain.invoke("곰은 무엇을 좋아하나요?")

'문서에 따르면, **곰은 꿀**을 좋아합니다.'

In [6]:
from langchain_community.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

prompt = ChatPromptTemplate.from_template(
    "{topic}에 대한 간단한 농담을 해줘."
)
output_parser = StrOutputParser()
model = ChatOllama(model=os.getenv("OPENAI_DEFAULT_MODEL"))
chain = (
    {"topic": RunnablePassthrough()} 
    | prompt
    | model
    | output_parser
)

for chunk in chain.stream("ice cream"):
    print(chunk, end="", flush=True)

물론이죠! 여기 아이스크림에 대한 간단한 농담이 있습니다:

왜 아이스크림이 학교에서 인기 있는 아이가 되고 싶어 할까요?

그건 항상 친구들 사이에서 '냉'하게 대우받지 않기 위해서죠! (Ice cream은 영어로 '아이스크림'이자 '차가운'이라는 단어의 더블 미닝을 활용한 농담입니다.)

In [7]:
chain.batch(["아이스크림", "스파게티", "만두"])

# stream: 입력에 대해 청크를 스트리밍
# invoke: 입력에 대해 체인을 호출
# batch: 입력목록에 대해 체인을 배치로 호출

['물론이죠! 여기 아이스크림에 대한 간단한 농담이 있습니다:\n\n왜 아이스크림 콘이 학교에서 좋은 학생으로 알려져 있을까요?\n\n그건 항상 자신의 한계를 \'섞\'지 않기 때문이죠! (Ice cream cones are known for their mix of flavors, 하지만 이 문맥에서는 "한계를 넘지 않는다"는 의미로 사용되었습니다.)',
 '물론이죠! 여기 스파게티에 대한 간단한 농담이 있습니다:\n\n"왜 스파게티가 학교에서 인기 있는 걸까요? 왜냐하면 모두가 파스타 친구들이니까요!"\n\n즐겁게 웃으실 수 있기를 바랍니다!',
 '물론이죠! 여기 만두에 대한 간단한 농담이 있습니다:\n\n"왜 만두는 항상 행복해 보일까요? 그 이유는 속이 다 채워져서 그렇답니다!"\n\n재미있게 웃으셨기를 바랍니다!']

In [8]:
await chain.ainvoke("ice cream")

# astream: 비동기로 입력에 대해 청크를 스트리밍
# ainvoke: 비동기로 입력에 대해 체인을 호출
# abatch: 비동기로 입력목록에 대해 체인을 배치로 호출

'물론이죠! 여기 아이스크림에 대한 간단한 농담이 있습니다:\n\n왜 아이스크림이 학교에서 인기 있는 아이가 되고 싶어 했을까요?\n\n그건 너무 달콤해서 모두가 그를 원했거든요! 🍦😄'

In [9]:
await chain.abatch(
    ["아이스크림", "스파게티", "만두"]
)

['물론이죠! 여기 아이스크림에 대한 간단한 농담이 있습니다:\n\n왜 아이스크림이 학교에서 인기 있는 아이가 되고 싶어 할까요?\n\n그 이유는, 모두가 그를 좋아하기 때문이죠! (특히 여름에는!)\n\n재미있게 즐기세요!',
 '물론이죠! 여기 스파게티에 대한 간단한 농담이 있습니다:\n\n왜 스파게티가 학교에서 인기 있는 아이가 되고 싶어 했을까요?\n\n그건 바로, 모두가 그를 "파스타"라고 불렀기 때문이죠! 😆',
 '물론이죠! 여기 만두에 대한 간단한 농담이 있습니다:\n\n"왜 만두는 항상 행복해 보일까요? 그 이유는 가득 찬 속 때문이죠!"\n\n재미있게 즐기시길 바랍니다!']

In [10]:
chain.input_schema.schema()

/tmp/ipykernel_44378/3226659032.py:1: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  chain.input_schema.schema()


{'properties': {'root': {'title': 'Root'}},
 'required': ['root'],
 'title': 'RunnableParallel<topic>Input',
 'type': 'object'}

In [11]:
chain.output_schema.schema()

{'title': 'StrOutputParserOutput', 'type': 'string'}